In [ ]:
import socketio
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.api.pset
import ifcopenshell.util.element
import pandas as pd

# Create a Socket.IO client
sio = socketio.Client()
d = {}

# Define event handlers
@sio.event
def connect():
    print("Connected to the server.")

@sio.event
def disconnect():
    print("Disconnected from the server.")

@sio.event
def user_opens(data):
    print(data)
    print(data['model_path'])
    d['model_path'] = data['model_path']
    d['ifc'] = ifcopenshell.open(data['model_path'])
    print('user_opens', d['ifc'])

def get_psets(e): return ifcopenshell.util.element.get_psets(e, psets_only=True)
def get_qtos(e): return ifcopenshell.util.element.get_psets(e, qtos_only=True)

@sio.on('highlighting')
def on_event(data):
    print("Data received:", data)
    d['guids'] = data['guids']
    ifc = d['ifc']
    for guid in d['guids']:
        e = ifc.by_guid(guid)
        psets = get_psets(e)
        for key, pset in dict(sorted(psets.items())).items():
            if "Common" in key:
                print(key, pd.DataFrame.from_dict(pset, 'index'))
        qtos = get_qtos(e)
        for k, qto in dict(sorted(qtos.items())).items():
            if "Qto" in k:
                print(k, pd.DataFrame.from_dict(qto, 'index'))

# Change the url of your app
cloud_run_url = 'http://127.0.0.1:8000/' # This is local hosted url
# cloud_run_url = 'https://pyijs-xxxxxxxxxxxx.europe-west6.run.app' # This could be the url of your deployed app on google cloud

# Connect to the Flask-SocketIO server
sio.connect(cloud_run_url)
sio.emit('feeds', 'Hello from Jupyter coding frontend!')

# Keep the client running to listen for events
sio.wait()